In [74]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END, MessagesState
from dotenv import load_dotenv
from langchain_core.tools import tool
import requests

In [75]:
load_dotenv()
model = ChatOpenAI()

In [76]:
@tool
def get_weather(query:str):
    """This returns the weather for the given location."""
    API_KEY = "48bf70709fa3413886855137251307"

    prompt1 = f"Which location is this request for: {query}?"
    location = model.invoke(prompt1).content

    response = requests.get(f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={location}")
    data = response.json()

    prompt = f"Summarize the weather details from the following: {data}"
    response = model.invoke(prompt).content

    return response

In [77]:
tool_node = ToolNode([get_weather])

model_with_tools = model.bind_tools([get_weather])

In [78]:
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


In [79]:
builder = StateGraph(MessagesState)

In [80]:
# Define the two nodes we will cycle between
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_node)

In [81]:
builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")


In [82]:
graph = builder.compile()

In [85]:
query = "What is the weather today in Delhi?"

In [86]:
result = graph.invoke({"messages": [{"role": "user", "content": query}]})

final_message = result["messages"][-1]
print(final_message.content)

The current weather in Delhi is 32.1°C with moderate to heavy rain accompanied by thunder. The wind is coming from the east at 7.9 km/h with a gust of 10.3 km/h. The humidity is at 67%, and the visibility is 2.5 km. The weather condition is expected to continue with a cloud cover of 100%. The "feels like" temperature is 34.0°C, with a heat index of 39.6°C.
